# [BLOCK-T284] - M2 inclinometer reading source - switch test

[BLOCK-T284]: https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/v2/testCase/BLOCK-T284/testScript

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import os

from matplotlib import pyplot as plt

from lsst.ts.block.utils import build_configuration_schema
from lsst.ts.observing import ObservingBlock, ObservingScript

In [ ]:
name = "BLOCK-T284"
program = "BLOCK-T284"
reason = "LVV-T3019"
constraints = []

output_folder = "output_blocks"

In [ ]:
properties = {
    "azimuth": {
        "description": "Start Azimuth position",
        "type": "number",
        "default": -32,
    },
    "ignore": {
        "description": "Name of the CSCs we want to ignore",
        "type": "array",
        "default": ["mtaos", "mtdome", "mtdometrajectory", "mtrotator"],
    },
}

block_number = name.split("-")[-1]
configuration_schema = build_configuration_schema(block_number, properties)
print(configuration_schema)

In [ ]:
sleep = ObservingScript(name="sleep.py", standard=True, parameters=dict(sleep_for=5))

move_p2p_1 = ObservingScript(
    name="maintel/move_p2p.py",
    standard=True,
    parameters=dict(
        az="$azimuth",
        el=84,
        move_timeout=2700,
        ignore="$ignore",
    ),
)

move_p2p_2 = ObservingScript(
    name="maintel/move_p2p.py",
    standard=True,
    parameters=dict(
        az="$azimuth",
        el=70,
        move_timeout=2700,
        ignore="$ignore",
    ),
)

standby_csc = ObservingScript(
    name="set_summary_state.py",
    standard=True,
    parameters=dict(data=[["MTM2", "STANDBY"]]),
)

disable_csc = ObservingScript(
    name="set_summary_state.py",
    standard=True,
    parameters=dict(data=[["MTM2", "DISABLED"]]),
)

enable_csc = ObservingScript(
    name="set_summary_state.py",
    standard=True,
    parameters=dict(data=[["MTM2", "ENABLED"]]),
)

set_inclination_source_m2 = ObservingScript(
    name="run_command.py",
    standard=True,
    parameters=dict(
        component="MTM2",
        cmd="selectInclinationSource",
        parameters=dict(source=1, maxDifference=2, enableComparison=False),
    ),
)

set_inclination_source_mtmount = ObservingScript(
    name="run_command.py",
    standard=True,
    parameters=dict(
        component="MTM2",
        cmd="selectInclinationSource",
        parameters=dict(source=2, maxDifference=5),
    ),
)

close_loop = ObservingScript(
    name="maintel/m2/enable_closed_loop.py", standard=True, parameters=dict()
)

open_loop = ObservingScript(
    name="maintel/m2/disable_closed_loop.py", standard=True, parameters=dict()
)


scripts = [
    move_p2p_1,
    sleep,
    disable_csc,
    sleep,
    standby_csc,
    sleep,
    disable_csc,
    sleep,
    set_inclination_source_mtmount,
    sleep,
    enable_csc,
    sleep,
    move_p2p_2,
    sleep,
    move_p2p_1,
    sleep,
    disable_csc,
    sleep,
    standby_csc,
    sleep,
    disable_csc,
    sleep,
    set_inclination_source_m2,
    sleep,
    enable_csc,
    sleep,
    move_p2p_2,
    sleep,
    move_p2p_1,
]

In [ ]:
block = ObservingBlock(
    name=name,
    program=program,
    scripts=scripts,
    constraints=constraints,
    configuration_schema=configuration_schema,
)

In [ ]:
block.model_dump_json(indent=2)

os.makedirs(output_folder, exist_ok=True)
output_path = f"{output_folder}/{name}.json"

with open(output_path, "w") as file:
    file.write(block.model_dump_json(indent=2))